In [4]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import html5lib
import time
from bs4 import BeautifulSoup
from selenium import webdriver
from dotenv import load_dotenv

# uses env file to store LayupList login information
load_dotenv()

LAYUPLIST_EMAIL = os.getenv('LAYUPLIST_EMAIL')
LAYUPLIST_PASSWORD = os.getenv('LAYUPLIST_PASSWORD')

# reads in course and department catalog
course_db = pd.read_csv ('../data/course_db.csv')
course_code_db = pd.read_csv ('../data/course_code_db.csv')

# logs into layuplist
driver = webdriver.Chrome('/Users/sarakay/Downloads/chromedriver')  # Optional argument, if not specified will search path.
driver.get('https://www.layuplist.com/accounts/login/');


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:21: DeprecationWarning: executable_path has been deprecated, please pass in a Service object


In [5]:
driver.find_element_by_id("login-email").send_keys(LAYUPLIST_EMAIL)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


In [6]:
driver.find_element_by_id("login-password").send_keys(LAYUPLIST_PASSWORD)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  """Entry point for launching an IPython kernel.


In [7]:
driver.find_element_by_xpath("/html/body/div/div[1]/div/div/div/div[5]/div/form/input[4]").click() 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  """Entry point for launching an IPython kernel.


In [ ]:
import re

driver.get('https://www.layuplist.com/departments')

# goes through each department name
for depart in course_code_db['Code']: 
    
    # does to page on layuplist containing department's courses
    URL = 'https://www.layuplist.com/search?q=' + depart
    driver.get(URL)
    
    # gets courses in department
    course_names = course_db[course_db['Course'].str.startswith(depart)]['Course']
    
    # goes to each course 
    for departCourse in course_names:
        driver.get(URL)
        print(departCourse)
        
        try:
            driver.find_element_by_partial_link_text(departCourse).click()
            soup = BeautifulSoup(driver.page_source)
            
            # gets number of pages of reviews to scrape
            maxNum = 1
            numRange = soup.findAll('ul')
            num = []
            urlStarter = driver.current_url
            for numbers in numRange:
                if 'of' in numbers.text:
                    num = re.findall('[0-9]+', numbers.text)
                    maxNum = int(num[1])
                    
            # list to store each page of reviews
            listToDF = []
            for i in range(1, maxNum + 1):
                newURL = urlStarter + '?page=' + str(i)
                driver.get(newURL)
                soup = BeautifulSoup(driver.page_source)

                paragraphs = soup.findAll('td')
                
                # puts review text into list
                for paragraph in paragraphs:
                    if paragraph.b != None:
                        listToDF.append(paragraph.text)
                        
            # creates dataframe from list of reviews
            coursedf = pd.DataFrame(listToDF, columns=['courseReviewRaw'])
            
            # labels which course review came from
            coursedf['Course'] = departCourse
            complete_df_list.append(coursedf)
        except:
            continue

In [41]:
complete_df_list

[                                     courseReviewRaw  \
 0  \n08W\n with Lindsay F.:\n                    ...   
 1  \n08W\n with Lindsay F.:\n                    ...   
 2  \n08W\n with Lindsay F.:\n                    ...   
 3  \n07W\n with Ayo Coly:\n                      ...   
 4  \n07W\n with Ayo Coly:\n                      ...   
 5  \n05S\n with William W.:\n                    ...   
 6  \n05S\n with William W.:\n                    ...   
 7  \n01F\n with Ifi Amadiume:\n                  ...   
 
                         Course  
 0  AAAS007: First Year Seminar  
 1  AAAS007: First Year Seminar  
 2  AAAS007: First Year Seminar  
 3  AAAS007: First Year Seminar  
 4  AAAS007: First Year Seminar  
 5  AAAS007: First Year Seminar  
 6  AAAS007: First Year Seminar  
 7  AAAS007: First Year Seminar  ,
                                      courseReviewRaw  \
 0  \n20F\n with Adedoyin Ogunfeyimi And Clifford ...   
 1  \n20F\n with Adedoyin Ogunfeyimi And Clifford ...   
 
     

In [92]:
layuplistreviews = pd.concat(complete_df_list)
layuplistreviews

courseReviewRaw  \
0   \n08W\n with Lindsay F.:\n                    ...   
1   \n08W\n with Lindsay F.:\n                    ...   
2   \n08W\n with Lindsay F.:\n                    ...   
3   \n07W\n with Ayo Coly:\n                      ...   
4   \n07W\n with Ayo Coly:\n                      ...   
..                                                ...   
6   \n09S\n with Jennifer Brooke:\n               ...   
7   \n09S\n with Jennifer Brooke:\n               ...   
8   \n09S\n with Jennifer Brooke:\n               ...   
9   \n09S\n with Jennifer Brooke:\n               ...   
10  \n09S\n with Jennifer Brooke:\n               ...   

                                   Course  
0             AAAS007: First Year Seminar  
1             AAAS007: First Year Seminar  
2             AAAS007: First Year Seminar  
3             AAAS007: First Year Seminar  
4             AAAS007: First Year Seminar  
..                                    ...  
6   WRIT043: The Written Judicial Opinion  
7   WRIT043: The Written Judicial Opinion  
8   WRIT043: The Written Judicial Opinion  
9   WRIT043: The Written Judicial Opinion  
10  WRIT043: The Written Judicial Opinion  

[29030 rows x 2 columns]

In [93]:
layuplistreviews.to_csv ('../data/layuplist_reviews.csv', index = False, header=True)